# Technical note: CVODES simulation and initial values

A `myokit.Model` defines an initial value problem.

$$\dot{y} = f(y, p, u(t), t)$$

$$y(t = 0) = y_0$$

where $y$ is the state, $p$ are the parameters of interest, $u(t)$ are inputs to the model, and $t$ is time.

**Note**: It does not specify constraints on $y$ and $p$, e.g. $y_i = [Ca^{2+}]_i \geq 0$ or $p_j = g_\text{Kr} \geq 0$. Instead, it is assumed that the trajectory starting at $y_0$ will stay within the biologically realistic range. The idea that $f$ is a good model only for certain values of $y$ is the reason that models are deemed "incomplete" until the modeller has added an initial condition.

A model can contain any number of "intermediary variables" (or "auxilliary functions"), for example currents.
These are calculated from the state, and used in the calculation of $f$, so for performance reasons it makes sense to calculate them just once (in an "intermediary" step).
Mathematically, it's easier to ignore that detail and think of them as "outputs" (or "derived quantities"), entirely seperate from $f$:

$$z = g(y, p, t, u(t))$$

### Myokit 1.34: Initial values as functions of $p$

In [Myokit 1.34.0](https://github.com/myokit/myokit/releases/tag/v1.34.0), it became possible to write $y_0$ as a function:

$$y(t = 0) = h(p)$$

## Simulations

A `myokit.Simulation` integrates an initial value problem to any time $t > 0$.

It can also calculate sensitivities of outputs with respect to parameters:

$$\frac{\partial{z_i}}{\partial{p_j}}$$

To do so, it internally calculates the full set of state sensitivities, by **solving another initial value problem**:

$$s = \frac{\partial{y}}{\partial{p}}$$

for all states and all parameters of interest, and then calculating $\frac{\partial z}{\partial p}$ using equations derived symbollically from $f$.

### Myokit 1.36: No more explicit sensitivities w.r.t. $y_0$

Before Myokit 1.34, the initial state could not depend on parameters, and so the only way to get the sensitivity of an output w.r.t. an initial condition, was through a seperate mechanism (using `InitialValue` objects or the `init(x)` syntax).
After 1.34, the simulation still treated $y_0$ as a scalar, and ignored information from $h$.
Starting with version 1.36 the simulation will instead symbolically derive sensitivities of $h$ w.r.t. $p$, and so a separate mechanism is no longer required.

For example, given a current $I_\text{Kr}$ with a state variable $n$, the old way to calculate a sensitivity w.r.t. $n(t=0)$
$$\frac{\partial I_\text{Kr}}{\partial n(t=0)}$$
was
```
sim = myokit.Simulation(model, sensitivities=[['ikr.IKr'], ['init(ikr.n)']])
```
In the new syntax, you can define $n(t = 0) = \texttt{ikr.n0}$ and use
```
sim = myokit.Simulation(model, sensitivities=[['ikr.IKr'], ['ikr.n0']])
```
and so the `init` syntax can be removed.

## The initial "state sensitivities"

Myokit does not provide a way to set the "initial state sensitivites":

$$s_0 = \left.\frac{\partial y}{\partial p}\right\rvert_{t=0}$$

There are two reasons for this:

1. The selection of "parameters of interest" is done at simulation time, so the matrix above is part of a `myokit.Simulation`, not part of `myokit.Model`.
2. In cardiac cell models, which can easily have 40 states, it would be really ardous to make the user write this out.

So what do we do instead? Three use-cases are shown below.

### Use case 1: A toy example, where $h(p) = y(t = 0)$ is taken as truth

First, let's assume that **$y(t = 0)$ is fully specified by $h(p)$, and that the system has no history before $t = 0$**.

This means that all dependence of $y(t = 0)$ on $p$ is captured in $h$, and any other initial state sensitivites should be 0.
For example, given a 2-state model with initial conditions

```
[[model]]
# Initial conditions
system.a = 0
system.b = 7 * system.c

[system]
c = 2
dot(a) = ...
```

and a simulation
```
sim = myokit.Simulation(model, sensitivities=[[...], ['system.c']])
```
we have $p = [\texttt{system.c}]$ and initial state sensitivities
$$
\left.\frac{\partial y}{\partial p}\right\rvert_{t=0} = \begin{bmatrix}0\\7\end{bmatrix}
$$

For most use cases, we except many states and parameters, but very few parameter-dependent initial conditions, so that the matrix will consist largely of zeroes.

### Use case 2: Starting from a limit cycle

Next we consider an action potential model, used to simulate the AP of an isolated myocyte driven by a periodic stimulus.

**Now we think of h(p) less as a definition, and more as a snapshot! The system definitely has a history, and so the initial state sensitivities should be set accordingly.**

Given the periodic forcing signal, we might assume that the system will converge to a periodic orbit (or limit cycle), and that we can approximate this by running a long simulation:

1. Start from a physiologically feasible $y_0$, leave $\left.\frac{\partial y}{\partial p}\right\rvert_{t=0}$ set to 0, and run for $N$ periods.
2. Take the new simulation state $y(t)$ to be your $y_0$, and the $s(t)$ to be your $s_0$, then redefine the current time as being 0.

This is what Myokit's `Simulation.pre()` does.

What about sensitivity w.r.t. initial conditions in this case?
If we're assuming that we are at a limit cycle, then they should be zero.
(If the system has multiple limit cycles or is chaotic, this doesn't mean that the starting point doesn't matter, just that the derivatives are not the appropriate way to look at this).

So for this use-case $h$ is not a very useful mechanism!
Instead, we must take care to first obtain a sensible $y_0$ and $s_0$.

#### What if we are still interested in $\partial/\partial y_0$

I'm not sure why you would be, but the way to go about this would be to:

1. Obtain $s_0$ from the simulation, after pre-pacing
2. Find the indices in $\partial y / \partial p$ that you want to modif, that is, the index of the state in the state vector, and the index of your parameter in the parameter vector. Presumably there are very small numbers at those entries now, because the effect of the original initial conditions has gone away. Set them to 1 and pass the updated $s_0$ back to the simulation.

This requires a method `Simulation.set_state_sensitivities`, that has not yet been implemented.

### Use case 3: Starting from a steady-state

Finally let's consider a HH-type ion current model, used in a simulation of a voltage-clamp experiment.
Here we assume that the system has a steady-state at a particular voltage (part of $u(t)$), and that we can either calculate this state directly from the equations, or that we can simply hold the system at a voltage for some time before starting the "real" simulation.

The strategy of simulating a long time at a fixed voltage is similar to the previous use case.
For the strategy where we calculate and insert, we need two things:

1. A way to calculate $s_0$. This could be handled by [future versions](https://github.com/myokit/myokit/issues/380) of the `HHModel` class, similar to the current method [HHModel.steady_state](https://myokit.readthedocs.io/en/stable/api_library/hh.html#myokit.lib.hh.HHModel.steady_state)
2. A method `Simulation.set_state_sensitivities`.

For Markov models we may be able to do something similar, but we could always resort to long pre-simulations instead.

#### Intermezzo: Sensitivities w.r.t. protocol parameters?

For thise cases, it might also make sense to have $h$ be a function of the input $u(t)$.
However, this can probably be worked around by adding equations to the model that manipulate $u(t)$, such that the parameters of interest become part of $p$.
For example:
```
[voltage_clamp]
dimensionless_clamp = 0
    bind pace
offset = -80 [mV]
    in [mV]
scaling = 120 [mV]
    in [mV]
V = offset + scaling * dimensionless_clamp
    in [mV]
```

used with a protocol that starts at a _relative_ potential of 0, this would allow model parameter `offset` to be treated as the holding potential, and so we can calculate sensitivities with respect to it.

## Changing things during the simulation

The `Simulation` class has various `set_x()` methods, that let you change the system, **thus breaking away from the initial value problem set by the Model**.

In the following list, methods _in italics_ are not implemented yet:

- set_time(), set_state(), and _set_state_sensitivities()_ set $t$, $y$ and $s$.
- set_default_state() and _set_default_state_sensitivities()_ set a "default" $y$ and $s$ that can be returned to by calling `reset()`.
- set_constant() changes the value of a model constant or parameter.

The last method in particular creates a little room for ambiguity:

#### Set_constant

If we have a $y_0 = h(p)$, then should the state sensitivities (or the default state sensitivities) in a simulation change when we change $p$?

Three mechanisms were considered:

1. $y_0$ is rederived whenever $p$ changes
2. $y_0$ is rederived when $p$ changes, but only after calling `reset()`
3. $y_0$ is never rederived

The first and second create all sorts of problems.
Some complicated solutions may be found for this, but would require users to understand them, potentially making life harder for all users with a very small pay-off for niche use cases (pre-pacing is almost always the answer!).

#### Setting (default) state and state sensitivities

A similar but perhaps less obvious problems arises for set_state() and set_state_sensitivities, because $h(p)$ implies a relationship between $s_0$ and $y_0$. What does it mean when a user changes $y_0$ but not $s_0$?

#### Conclusion

The general principle will stay much the same as it currently is:

1. A model defines an initial value problem.
2. A simulation is created as a representation of this IVP, but can then be changed.

In other words, while a simulation starts at $t=0, y=h(p)$,  anything the user does after that _changes_ the problem and can break the relationship $h(p)$ stated in the `Model`.

Implementation:

- When a `Simulation` is created
  - time is set to zero
  - the state and default state are set to the evaluation of $h(p)$ for the current $p$
  - the state sensitivities (and defaults) are set to the evaluation of $\partial h / \partial p$ for the current $p$
- After this, $y$, $s$ and their default values are numbers